In [1]:
import requests 
import json
from bs4 import BeautifulSoup as bs
from pprint import pprint
import pandas as pd

pd.set_option('display.max_rows', None)

# Задание 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

1. Наименование вакансии.
2. Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
3. Ссылку на саму вакансию.
4. Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [2]:
# Введите интересующую вакансию, регион и кол-во страниц
vacancy_name = 'game analyst'
vacancy_region = 1
number_of_pages = 5

In [3]:
url = 'https://hh.ru'
params = {'clusters': 'true',
          'area': vacancy_region,
          'ored_clusters': 'true',
          'enable_snippets': 'true',
          'salary': None,
          'text': vacancy_name,
          'page': 0}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

vacancies_list = []

while params['page'] < number_of_pages:
    response = requests.get(url + '/search/vacancy', params=params, headers=headers)
    dom = bs(response.text, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item'})

    if response.ok and vacancies:
        
        for vacancy in vacancies:
            vacancy_data = {}
            info = vacancy.find('a', {'class': 'bloko-link'})
            name = info.text
            company = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info-company'}).text
            link = info['href']
            site = url
            
            try:
                salary = vacancy.find('div', {'class': 'vacancy-serp-item__sidebar'}).text
                i = salary.split()
                if 'от' in salary:
                    min_salary = int(i[1] + i[2])
                    max_salary = None
                    fix_salary = None
                    currency = i[3]
                elif 'до' in salary:
                    min_salary = None
                    max_salary = int(i[1] + i[2])
                    fix_salary = None
                    currency = i[3]
                elif ' - ' in salary:
                    min_salary = int(i[0] + i[1])
                    max_salary = int(i[3] + i[4])
                    fix_salary = None
                    currency = i[5]
                else:
                    min_salary = None
                    max_salary = None
                    fix_salary = int(i[0] + i[1])
                    currency = i[2]
            except:
                #salary = None
                min_salary = None
                max_salary = None
                fix_salary = None
                currency = None
            
            vacancy_data['vacancy'] = name
            vacancy_data['company'] = company
            vacancy_data['link'] = link
            vacancy_data['source'] = site
           # vacancy_data['min_salary'] = min_salary
           # vacancy_data['max_salary'] = max_salary
           # vacancy_data['fix_salary'] = fix_salary
           # vacancy_data['currency'] = currency
            vacancies_list.append(vacancy_data)
        print(f"Обработана {params['page']} страница")
        params['page'] += 1
    else:
        break
        
result = pd.DataFrame(vacancies_list)

#pprint(vacancies_list)

Обработана 0 страница
Обработана 1 страница
Обработана 2 страница
Обработана 3 страница


In [4]:
result

,vacancy,company,link,source
0,Game Analyst / Игровой Аналитик,PlayQuantum,https://hh.ru/vacancy/48725827?from=vacancy_se...,https://hh.ru
1,Game Analyst,Rightway Games,https://hh.ru/vacancy/49220216?from=vacancy_se...,https://hh.ru
2,Игровой аналитик / Game Analyst (PC/Console),СберИгры,https://hh.ru/vacancy/48375196?from=vacancy_se...,https://hh.ru
3,Junior Game Analyst,NEXTRP,https://hh.ru/vacancy/48809730?from=vacancy_se...,https://hh.ru
4,Product Analyst (SQL+R/Python),RJ Games,https://hh.ru/vacancy/47608292?from=vacancy_se...,https://hh.ru
5,Game Producer (Hyper Casual Games),Храмцова Полина,https://hh.ru/vacancy/48357502?from=vacancy_se...,https://hh.ru
6,Product Analyst (f2p mobile games),ZeptoLab,https://hh.ru/vacancy/49030653?from=vacancy_se...,https://hh.ru
7,Senior/Lead Data Analyst,VOKI Games,https://hh.ru/vacancy/49209976?from=vacancy_se...,https://hh.ru
8,Геймдизайнер / Middle Game Designer,Mehen Games,https://hh.ru/vacancy/48721426?from=vacancy_se...,https://hh.ru
9,Senior Game Monetization Designer,RJ Games,https://hh.ru/vacancy/46987932?from=vacancy_se...,https://hh.ru
